In [1]:
from transformers import AutoTokenizer
from transformers import MistralForCausalLM,MistralConfig
import torch

/usr/scratch/haokang/anaconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_ID = "mistralai/Mistral-7B-v0.1"
model_kwargs = {}
model_kwargs["torch_dtype"] = torch.float16
model_kwargs["device_map"] = "auto"
model_kwargs["token"] = None
model_kwargs["cache_dir"] = "../cache"

In [3]:
config = MistralConfig.from_pretrained(
    MODEL_ID,
    use_auth_token=True,
    token=None,
    use_flash_attn=False,
)
model = MistralForCausalLM.from_pretrained(
    MODEL_ID,
    config=config,
    **model_kwargs,
    # compress_config=compress_config,
)
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    token=None,
    padding_side="left",
    model_max_length=256,
    use_fast=False,
    cache_dir="../cache",
)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

/usr/scratch/haokang/anaconda3/envs/test/lib/python3.10/site-packages/transformers/configuration_utils.py:508: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.35s/it]


1

In [10]:
generate_kwargs = dict(
    return_dict_in_generate=True,
    max_length=None,
    max_new_tokens=256,
    output_scores=True,
    pad_token_id=tokenizer.eos_token_id,
    use_cache=True,
)


generate_kwargs["do_sample"] = False
generate_kwargs["temperature"] = None
generate_kwargs["top_k"] = None
generate_kwargs["top_p"] = None

In [13]:
prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt,prompt], return_tensors="pt").to("cuda")
print(model_inputs["input_ids"].shape)
generated_ids = model.generate(**model_inputs, **generate_kwargs)
generations = tokenizer.batch_decode(
    generated_ids.sequences[:, model_inputs.input_ids.shape[1] :],
    skip_special_tokens=True,
)
print(generations)

torch.Size([2, 6])


/usr/scratch/haokang/anaconda3/envs/test/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/scratch/haokang/anaconda3/envs/test/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/scratch/haokang/anaconda3/envs/test/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:404: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


['mayonnaise. I love it on sandwiches, in salads, on burgers, in potato salad, on fries, in devilled eggs, on chicken, on fish, on pizza, on tacos, on baked potatoes, on hot dogs, on hamburgers, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sandwiches, on sand', 'mayonnaise. I love it on sandwiches, in salads, on burgers, in potato salad, on fries, in devilled eggs, on chicken, on fish, on pizza, on tacos, on baked potatoes, on hot dogs, on hamburgers, on sandwiches, on sandwiches, o